In [1]:
import numpy as np
import pickle

## Define the Tic Tac Toe Board

In [2]:
# Defining rows and columns
BOARD_ROWS = 3
BOARD_COLS = 3

## Function for the different states that we can take

In [3]:
class State:
    def __init__(self, p1, p2):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.boardHash = None
        # init p1 plays first
        self.playerSymbol = 1

In [4]:
# get unique hash of the current board state
def getHash(self):
    self.boardHash = str(self.board.reshape(BOARD_COLS*BOARD_ROWS))
    return self.boardHash

## Function to get the winner of the game

In [5]:
def winner(self):
    # row
    for i in range(BOARD_ROWS):
        if sum(self.board[i, :]) == 3:
            self.isEnd = True
            return 1
        if sum(self.board[i, :]) == -3:
            self.isEnd = True
            return -1
        
    # col
    for i in range(BOARD_COLS):
        if sum(self.board[:, i]) == 3:
            self.isEnd = True
            return 1
        if sum(self.board[:, i]) == -3:
            self.isEnd = True
            return -1
        
    # diagonal
    diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)])
    diag_sum2 = sum([self.board[i, BOARD_COLS--i-1] for i in range(BOARD_COLS)])
    if diag_sum == 3:
        self.isEnd = True
        return 1
    if diag_sum == -3:
        self.isEnd = True
        retirn -1